In [2]:
from montecarlo import MonteCarlo
from scipy.stats import norm
import numpy as np

Analytical value of $\delta$: 

In [2]:
norm.cdf(1 / (0.2 * np.sqrt(1)) * (np.log(100 / 99) + (0.06 + ((0.2 ** 2) / 2)) * 1)) - 1

-0.3262644882651039

## Bump and evaluate
Different seed:

In [22]:
n = 100
for s in [100, 1000, 10000]:
    v_bumped = np.empty((n, s))
    v_unbumped = np.empty((n, s))
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v_bumped[i] = MonteCarlo(100+e, 0.06, 1, 99, 365, 0.2).run_immediately(s)
            v_unbumped[i] = MonteCarlo(100-e, 0.06, 1, 99, 365, 0.2).run_immediately(s)
        deltas = (np.mean(v_bumped, axis=1) - np.mean(v_unbumped, axis=1)) / (2*e)
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=-0.1890424901867616 (0.05295903993537422)
e=0.1, s=100: delta=-1.6239674738996037 (0.5688426961048638)
e=0.01, s=100: delta=-3.4482360371714065 (5.500512333320702)
e=1, s=1000: delta=-0.341479094793473 (0.01726405230959819)
e=0.1, s=1000: delta=-0.4030641740800421 (0.17741777753587623)
e=0.01, s=1000: delta=-1.4018734996021354 (1.7131912785303853)
e=1, s=10000: delta=-0.3220381452155043 (0.005429485712071461)
e=0.1, s=10000: delta=-0.30082081734047184 (0.04816665084058998)
e=0.01, s=10000: delta=-0.19388659476044487 (0.5843275034329891)


Same seed:

In [26]:
n = 100
for s in [100, 1000, 10000]:
    v_bumped = np.empty((n, s))
    v_unbumped = np.empty((n, s))
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v_bumped[i] = MonteCarlo(100+e, 0.06, 1, 99, 365, 0.2).run_immediately(s, seed=i)
            v_unbumped[i] = MonteCarlo(100-e, 0.06, 1, 99, 365, 0.2).run_immediately(s, seed=i)
        deltas = (np.mean(v_bumped, axis=1) - np.mean(v_unbumped, axis=1)) / (2*e)
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=-0.32574392546170267 (0.0038433439372400763)
e=0.1, s=100: delta=-0.32490441995316016 (0.003804892607429146)
e=0.01, s=100: delta=-0.3249791810719173 (0.0037846266863839815)
e=1, s=1000: delta=-0.3263072541496955 (0.0012457578871803048)
e=0.1, s=1000: delta=-0.32622489152119527 (0.0012641081454193245)
e=0.01, s=1000: delta=-0.3262377957948672 (0.001266926443723131)
e=1, s=10000: delta=-0.32607670886039336 (0.00038896014225508596)
e=0.1, s=10000: delta=-0.32599941342358413 (0.0003948609699928377)
e=0.01, s=10000: delta=-0.32601473838437206 (0.00039280615292039905)


## Delta digital option
Using bump and evaluate:

In [29]:
 n = 100
for s in [100, 1000, 10000]:
    v_bumped = np.empty((n, s))
    v_unbumped = np.empty((n, s))
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v_bumped[i] = MonteCarlo(100+e, 0.06, 1, 99, 365, 0.2, digital=True).run_immediately(s, seed=i)
            v_unbumped[i] = MonteCarlo(100-e, 0.06, 1, 99, 365, 0.2, digital=True).run_immediately(s, seed=i)
        deltas = (np.mean(v_bumped, axis=1) - np.mean(v_unbumped, axis=1)) / (2*e)
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=-0.018317320178213633 (0.000887639220027559)
e=0.1, s=100: delta=-0.015539114804140101 (0.0025091415428086764)
e=0.01, s=100: delta=-0.018835290671684962 (0.009273856047227275)
e=1, s=1000: delta=-0.018463293680919198 (0.0002491040963681217)
e=0.1, s=1000: delta=-0.01859984953828892 (0.0009112453349049742)
e=0.01, s=1000: delta=-0.015068232537347818 (0.0027497673888569134)
e=1, s=10000: delta=-0.018062101989612306 (8.259932746478783e-05)
e=0.1, s=10000: delta=-0.018444458390247543 (0.0002827714357158013)
e=0.01, s=10000: delta=-0.01662214401776199 (0.0008940752983179588)


Using Likelihood ratio:

In [30]:
n = 100
for s in [100, 1000, 10000]:
    v = np.empty((n, s))
    deltas = np.empty(n)
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            v[i] = MonteCarlo(100, 0.06, 1, 99, 365, 0.2, digital=True).run_immediately(s, seed=i)
            
            np.random.seed(i)
            epsilons = np.random.normal(size=s)
            deltas[i] = np.mean(v[i] * epsilons / (100 * 0.2 * np.sqrt(1)))
        
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=-0.018085876561628328 (0.0002705895709301)
e=0.1, s=100: delta=-0.018085876561628328 (0.0002705895709301)
e=0.01, s=100: delta=-0.018085876561628328 (0.0002705895709301)
e=1, s=1000: delta=-0.01815522588671591 (8.77926978222565e-05)
e=0.1, s=1000: delta=-0.01815522588671591 (8.77926978222565e-05)
e=0.01, s=1000: delta=-0.01815522588671591 (8.77926978222565e-05)
e=1, s=10000: delta=-0.018200035014475827 (2.607577012394789e-05)
e=0.1, s=10000: delta=-0.018200035014475827 (2.607577012394789e-05)
e=0.01, s=10000: delta=-0.018200035014475827 (2.607577012394789e-05)


Using pathwise:

In [18]:
n = 100
for s in [100, 1000, 10000]:
    v = np.empty((n, s))
    deltas = np.empty(n)
    for e in [1, 0.1, 0.01]:
        for i in range(n):
            stock_prices = np.array([100] * s, dtype=np.float64)
            stock_prices *= np.exp((0.06 - 0.5 * 0.2**2) + 0.2 *  np.random.normal(size=s))
            payoffs = norm.pdf(stock_prices, 99) * np.exp(-0.06)

            deltas[i] = np.mean(payoffs * stock_prices / 100)
        
        print(f'e={e}, s={s}: delta={np.mean(deltas)} ({np.std(deltas, ddof=1) / np.sqrt(n)})')

e=1, s=100: delta=0.018956056087610515 (0.0006757135134241316)
e=0.1, s=100: delta=0.01725403211426736 (0.0007364437321554505)
e=0.01, s=100: delta=0.017500785328692745 (0.0006062587348751807)
e=1, s=1000: delta=0.01823847349646597 (0.00021439862674964364)
e=0.1, s=1000: delta=0.018014408540463343 (0.0002192774211176674)
e=0.01, s=1000: delta=0.018491521264774838 (0.00023743120459168002)
e=1, s=10000: delta=0.018200700345436263 (6.389803363131993e-05)
e=0.1, s=10000: delta=0.01818272669249265 (6.904297019331926e-05)
e=0.01, s=10000: delta=0.018248780814510722 (7.122910945455674e-05)
